# Integrantes

- Gabriel de Carvalho Barros - 1698558  
- Henrique Melo Alves Martins - 1982661  
- João Victor da Silva - 6296342  
- Leticia Silverio - 5361635  
- Vitor Roggeri Cavicchiolli - 4687234  
- Rennys Rodrigues Cardoso - 3639077  

In [51]:
'''
from google.colab import drive
drive.mount('/content/drive')
'''

"\nfrom google.colab import drive\ndrive.mount('/content/drive')\n"

In [52]:
'''
import zipfile

with zipfile.ZipFile('dataset_00_sem_virg.zip', 'r') as zip_ref:
    zip_ref.extractall()
print("Arquivo extraído com sucesso."
''' 

'\nimport zipfile\n\nwith zipfile.ZipFile(\'dataset_00_sem_virg.zip\', \'r\') as zip_ref:\n    zip_ref.extractall()\nprint("Arquivo extraído com sucesso."\n'

# Arquivos de Sistema

In [53]:
%%writefile sistema.hpp

#ifndef SISTEMA_HPP
#define SISTEMA_HPP

#include <string>
#include <sys/stat.h>
#ifdef _WIN32
#include <windows.h>
#include <psapi.h>
#include <tchar.h>
#else
#include <sys/sysinfo.h>
#include <fstream>
#endif

using namespace std;

#define MB (1024 * 1024)

inline string extrair_nome_arquivo(const string& caminho_completo) {
    size_t pos = caminho_completo.find_last_of("/\\");
    if (pos == string::npos) return caminho_completo;
    return caminho_completo.substr(pos + 1);
}

inline size_t get_pico_uso_memoria_mb() {
#ifdef _WIN32
    PROCESS_MEMORY_COUNTERS info;
    GetProcessMemoryInfo(GetCurrentProcess(), &info, sizeof(info));
    return info.PeakWorkingSetSize / (1024 * 1024);
#else
    ifstream status("/proc/self/status");
    string linha;
    while (getline(status, linha)) {
        if (linha.find("VmHWM:") == 0) {
            size_t kb;
            sscanf(linha.c_str(), "VmHWM: %lu kB", &kb);
            return kb / 1024;
        }
    }
    return 0;
#endif
}

inline size_t get_memoria_total_mb() {
#ifdef _WIN32
    MEMORYSTATUSEX status;
    status.dwLength = sizeof(status);
    GlobalMemoryStatusEx(&status);
    return static_cast<size_t>(status.ullTotalPhys / MB);
#else
    struct sysinfo info;
    if (sysinfo(&info) != 0) return 1024;
    return info.totalram * info.mem_unit / MB;
#endif
}

inline size_t get_memoria_disponivel_mb() {
#ifdef _WIN32
    MEMORYSTATUSEX status;
    status.dwLength = sizeof(status);
    GlobalMemoryStatusEx(&status);
    return static_cast<size_t>(status.ullAvailPhys / MB);
#else
    struct sysinfo info;
    if (sysinfo(&info) != 0) return 1024;
    return info.freeram * info.mem_unit / MB;
#endif
}

inline size_t get_tamanho_arquivo_mb(const std::string& caminho) {
#ifdef _WIN32
    WIN32_FILE_ATTRIBUTE_DATA fileInfo;
    if (!GetFileAttributesExA(caminho.c_str(), GetFileExInfoStandard, &fileInfo)) {
        return 0;
    }

    ULARGE_INTEGER size;
    size.HighPart = fileInfo.nFileSizeHigh;
    size.LowPart = fileInfo.nFileSizeLow;
    return static_cast<size_t>(size.QuadPart / MB);
#else
    struct stat stat_buf;
    if (stat(caminho.c_str(), &stat_buf) != 0) return 0;
    return static_cast<size_t>(stat_buf.st_size / MB);
#endif
}

inline string get_nome_cpu() {
#ifdef _WIN32
    HKEY hKey;
    if (RegOpenKeyEx(HKEY_LOCAL_MACHINE,
        TEXT("HARDWARE\\DESCRIPTION\\System\\CentralProcessor\\0"),
        0, KEY_READ, &hKey) == ERROR_SUCCESS) {

        TCHAR nome_cpu[256];
        DWORD tamanho = sizeof(nome_cpu);
        if (RegQueryValueEx(hKey, TEXT("ProcessorNameString"), NULL, NULL, (LPBYTE)nome_cpu, &tamanho) == ERROR_SUCCESS) {
            RegCloseKey(hKey);
            return string(nome_cpu, nome_cpu + _tcslen(nome_cpu));
        }
        RegCloseKey(hKey);
    }
    return "Desconhecido";
#else
    ifstream cpuinfo("/proc/cpuinfo");
    string linha;
    while (getline(cpuinfo, linha)) {
        if (linha.find("model name") != string::npos) {
            return linha.substr(linha.find(":") + 2);
        }
    }
    return "Desconhecido";
#endif
}

#endif // SISTEMA_HPP


Overwriting sistema.hpp


# Arquivos de Leitura

In [54]:
%%writefile leitura.hpp

#ifndef LEITURA_HPP
#define LEITURA_HPP

#include <vector>
#include <string>
#include <fstream>
#include <sstream>
#include <iostream>

using namespace std;
using Linha = vector<string>;

#define MB (1024 * 1024)

// remove espaços, \r, \n e \t do início e fim da string
static inline string trim(const string& s) {
    size_t start = s.find_first_not_of(" \t\r\n");
    if (start == string::npos) return "";
    size_t end = s.find_last_not_of(" \t\r\n");
    return s.substr(start, end - start + 1);
}

// detecta se a string representa um numero (int/float)
inline bool eh_numero(const string& valor) {
    string val = trim(valor);
    if (val.empty()) return false;

    // remove aspas externas, se houver
    if (val.front() == '"' && val.back() == '"' && val.length() > 1)
        val = val.substr(1, val.length() - 2);

    bool ponto = false;
    size_t i = 0;
    if (val[0] == '-' || val[0] == '+') i++;

    for (; i < val.size(); ++i) {
        unsigned char c = static_cast<unsigned char>(val[i]);
        if (!isdigit(c)) {
            if (c == '.' && !ponto) {
                ponto = true;
            } else {
                return false;
            }
        }
    }
    return true;
}

inline void detectar_colunas_categoricas(const vector<string>& nomeColunas,
                                         const vector<vector<string>>& amostras,
                                         vector<int>& indicesAlvo) {

    size_t numLinhas = min<size_t>(amostras.size(), 5);

    for (size_t col = 0; col < nomeColunas.size(); ++col) {
        bool temAlfanumerico = false;

        // se em alguma das linhas existir valor “não vazio” e não-numérico, é categórica
        for (size_t lin = 0; lin < numLinhas; ++lin) {
            const string& raw = amostras[lin][col];
            string val = trim(raw);
            if (!val.empty() && val != "\"\"" && !eh_numero(val)) {
                temAlfanumerico = true;
                break;
            }
        }

        if (temAlfanumerico) {
            indicesAlvo.push_back(col);
        }
    }
}


inline void ler_cabecalho_e_amostras(const string& caminho,
                                     vector<string>& nomeColunas,
                                     vector<Linha>& amostras,
                                     int linhasAmostra = 5) {
    ifstream arquivo(caminho);
    if (!arquivo.is_open()) {
        cerr << "Nao foi possivel abrir o arquivo." << endl;
        exit(1);
    }

    string cabecalho;
    getline(arquivo, cabecalho);
    stringstream ss(cabecalho);
    string coluna;
    while (getline(ss, coluna, ',')) {
        nomeColunas.push_back(coluna);
    }

    string linha;
    while (linhasAmostra-- > 0 && getline(arquivo, linha)) {
        stringstream ssl(linha);
        string valor;
        Linha linhaDados;
        while (getline(ssl, valor, ',')) {
            linhaDados.push_back(valor);
        }
        amostras.push_back(linhaDados);
    }
    arquivo.close();
}


size_t calcular_tamanho_chunk_mb(size_t bytes_por_linha, size_t linhas_por_chunk) {
    return (bytes_por_linha * linhas_por_chunk) / MB;
}

size_t calcular_linhas_por_chunk(size_t memoria_disponivel_mb, size_t media_bytes_por_linha = 200) {
    size_t memoria_bytes = memoria_disponivel_mb * MB;
    size_t memoria_para_uso = memoria_bytes / 25; // Usa 10% da RAM disponível
    return memoria_para_uso / media_bytes_por_linha;
}

inline size_t calcular_bytes_por_linha_com_amostras(const vector<Linha>& amostras) {
    if (amostras.empty()) return 200;
    size_t total = 0;
    for (const auto& linha : amostras) {
        for (const auto& campo : linha) {
            total += campo.size();
        }
        total += linha.size() - 1; // vírgulas
    }
    return total / amostras.size();
}

#endif // LEITURA_HPP

Overwriting leitura.hpp


# Arquivos de Benchmark

In [55]:
%%writefile benchmark.hpp

#ifndef BENCHMARK_HPP
#define BENCHMARK_HPP

#include <string>
#include <vector>
#include <fstream>
#include <iostream>
#include <iomanip>
#include "sistema.hpp"
#include "leitura.hpp"

using namespace std;

struct BenchmarkInfo {
    string nome_arquivo;
    size_t tamanho_mb;
    size_t ram_disponivel_mb;
    size_t ram_total_mb;
    size_t bytes_por_linha;
    size_t linhas_por_chunk;
    double tamanho_chunk_mb;
    string cpu;
    int num_threads;
    int colunas_categoricas;
    double tempo_execucao;
};

inline BenchmarkInfo coletar_info_benchmark(const string& caminho_dataset,
                                            const vector<Linha>& amostras,
                                            int colunas_categoricas,
                                            int num_threads
                                            ) {
    BenchmarkInfo info;
    info.nome_arquivo = extrair_nome_arquivo(caminho_dataset);
    info.tamanho_mb = get_tamanho_arquivo_mb(caminho_dataset);
    info.ram_disponivel_mb = get_memoria_disponivel_mb();
    info.ram_total_mb = get_memoria_total_mb();
    info.bytes_por_linha = calcular_bytes_por_linha_com_amostras(amostras);
    info.linhas_por_chunk = calcular_linhas_por_chunk(info.ram_disponivel_mb, info.bytes_por_linha);
    info.tamanho_chunk_mb = calcular_tamanho_chunk_mb(info.bytes_por_linha, info.linhas_por_chunk);
    info.num_threads = num_threads;
    info.colunas_categoricas = colunas_categoricas;
    info.cpu = get_nome_cpu();
    return info;
}

inline void exibir_info_benchmark(const BenchmarkInfo& info) {
    cout << "\n===== INFOS =====" << endl;
    cout << "Nome do arquivo: " << info.nome_arquivo << endl;
    cout << "Tamanho do arquivo: " << info.tamanho_mb << " MB" << endl;
    cout << "RAM disponivel: " << info.ram_disponivel_mb << " MB" << endl;
    cout << "RAM total do sistema: " << info.ram_total_mb << " MB" << endl;
    cout << "Bytes por linha (estimado): " << info.bytes_por_linha << endl;
    cout << "Linhas por chunk: " << info.linhas_por_chunk << endl;
    cout << "Tamnho por chunk: " << info.tamanho_chunk_mb << " MB" << endl;

    cout << "Processador: " << info.cpu << endl;
    cout << "Cores (threads): " << info.num_threads << endl;
    cout << "Colunas categóricas detectadas: " << info.colunas_categoricas << endl;
    cout << "======================\n" << endl;
}

inline string formatar_tempo(double segundos) {
    int minutos = static_cast<int>(segundos) / 60;
    int seg = static_cast<int>(segundos) % 60;
    ostringstream oss;
    oss << minutos << "m " << seg << "s";
    return oss.str();
}

inline void salvar_info_benchmark_csv(const BenchmarkInfo& info, const string& caminho_saida) {
    bool arquivo_existe = ifstream(caminho_saida).good();
    ofstream arq(caminho_saida, ios::app);
    if (!arq.is_open()) {
        cerr << "Erro ao salvar informacoes no CSV." << endl;
        return;
    }

    if (!arquivo_existe) {
        arq << "nome_arquivo,nome_cpu,num_threads,ram_total_mb,ram_disponivel_mb,tamanho_arq_mb,bytes_por_linha,linhas_por_chunk,tamanho_chunk_mb,total_col_categoricas,tempo_execucao_segundos,tempo_formatado\n";
    }

    arq << info.nome_arquivo << ","
        << '"' << info.cpu << '"' << ","
        << info.num_threads << ","
        << info.ram_total_mb << ","
        << info.ram_disponivel_mb << ","
        << info.tamanho_mb << ","
        << info.bytes_por_linha << ","
        << info.linhas_por_chunk << ","
        << fixed << setprecision(2) << info.tamanho_chunk_mb << ","
        << info.colunas_categoricas << ","
        << fixed << setprecision(2) << info.tempo_execucao << ","
        << formatar_tempo(info.tempo_execucao) << "\n";

    arq.close();
}

#endif // BENCHMARK_HPP


Overwriting benchmark.hpp


# Executar OMP

In [56]:
%%writefile normalizador_omp.hpp

#ifndef NORMALIZADOR_OMP_HPP
#define NORMALIZADOR_OMP_HPP

#include <vector>
#include <string>
#include <unordered_map>
#include <unordered_set>
#include <fstream>
#include <sstream>
#include <omp.h>
#include <iostream>

#include <queue>
#include <map>
#include <thread>
#include <mutex>
#include <condition_variable>
#include <atomic>

using namespace std;
using Linha = vector<string>;

inline void construir_mapas_codificacao_por_chunks(const string& caminho,
                                                   const vector<string>& nomeColunas,
                                                   const vector<int>& indicesAlvo,
                                                   size_t linhas_por_chunk,
                                                   unordered_map<string, unordered_map<string, int>>& mapas,
                                                   unordered_map<string, int>& contadores) {
    ifstream arquivo(caminho);
    if (!arquivo.is_open()) {
        cerr << "Erro ao abrir o arquivo para processar chunks." << endl;
        exit(1);
    }

    string cabecalho;
    getline(arquivo, cabecalho); // descarta cabeçalho

    string linha;
    vector<Linha> chunk;

    while (getline(arquivo, linha)) {
        stringstream ss(linha);
        string valor;
        Linha linhaDados;
        while (getline(ss, valor, ',')) {
            linhaDados.push_back(valor);
        }
        chunk.push_back(linhaDados);

        if (chunk.size() >= linhas_por_chunk) {
            int num_threads = omp_get_max_threads();
            vector<unordered_map<string, unordered_map<string, int>>> mapasLocais(num_threads);

            #pragma omp parallel for schedule(dynamic)
            for (int i = 0; i < indicesAlvo.size(); i++) {
                int col = indicesAlvo[i];
                string nomeCol = nomeColunas[col];
                int tid = omp_get_thread_num();

                for (const auto& linhaChunk : chunk) {
                    if (col >= linhaChunk.size()) continue;
                    string valor = linhaChunk[col];
                    if (valor.empty()) continue;

                    mapasLocais[tid][nomeCol][valor];
                }
            }

            for (const auto& local : mapasLocais) {
                for (const auto& [coluna, mapa] : local) {
                    for (const auto& [valor, _] : mapa) {
                        if (mapas[coluna].find(valor) == mapas[coluna].end()) {
                            mapas[coluna][valor] = contadores[coluna]++;
                        }
                    }
                }
            }
            chunk.clear();
        }
    }

    if (!chunk.empty()) {
        int num_threads = omp_get_max_threads();
        vector<unordered_map<string, unordered_map<string, int>>> mapasLocais(num_threads);

        #pragma omp parallel for schedule(dynamic)
        for (int i = 0; i < indicesAlvo.size(); i++) {
            int col = indicesAlvo[i];
            string nomeCol = nomeColunas[col];
            int tid = omp_get_thread_num();

            for (const auto& linhaChunk : chunk) {
                if (col >= linhaChunk.size()) continue;
                string valor = linhaChunk[col];
                if (valor.empty()) continue;

                mapasLocais[tid][nomeCol][valor];
            }
        }

        for (const auto& local : mapasLocais) {
            for (const auto& [coluna, mapa] : local) {
                for (const auto& [valor, _] : mapa) {
                    if (mapas[coluna].find(valor) == mapas[coluna].end()) {
                        mapas[coluna][valor] = contadores[coluna]++;
                    }
                }
            }
        }
    }

    arquivo.close();
}

inline void gerar_arquivos_id_por_coluna(const unordered_map<string, unordered_map<string, int>>& mapas) {
    #pragma omp parallel for schedule(dynamic)
    for (int i = 0; i < mapas.size(); ++i) {
        auto it = next(mapas.begin(), i);
        const string& nomeCol = it->first;
        const auto& mapa = it->second;

        ofstream arquivo("ID_" + nomeCol + ".csv");
        if (!arquivo.is_open()) {
            #pragma omp critical
            cerr << "Erro ao criar arquivo ID_" << nomeCol << ".csv" << endl;
            continue;
        }

        arquivo << "COD,CONTEUDO\n";
        for (const auto& [conteudo, cod] : mapa) {
            arquivo << cod << "," << conteudo << "\n";
        }
        arquivo.close();
    }
}

inline vector<string> codificar_linhas_chunk(const vector<string>& chunk,
                                      const vector<string>& nomeColunas,
                                      const unordered_map<string, unordered_map<string, int>>& mapas) {
    vector<string> linhasCodificadas(chunk.size());
    #pragma omp parallel for
    for (int i = 0; i < chunk.size(); i++) {
        stringstream ss(chunk[i]);
        string valor;
        vector<string> campos;
        while (getline(ss, valor, ',')) {
            campos.push_back(valor);
        }
        stringstream codificada;
        for (int j = 0; j < campos.size(); j++) {
            const string& nomeCol = nomeColunas[j];
            if (mapas.count(nomeCol)) {
                codificada << mapas.at(nomeCol).at(campos[j]);
            } else {
                codificada << campos[j];
            }
            if (j < campos.size() - 1) codificada << ",";
        }
        linhasCodificadas[i] = codificada.str();
    }
    return linhasCodificadas;
}

inline void gerar_dataset_codificado(const string& caminho,
                                     const vector<string>& nomeColunas,
                                     const vector<int>& indicesAlvo,
                                     const unordered_map<string, unordered_map<string, int>>& mapas,
                                     size_t linhas_por_chunk) {
    ifstream entrada(caminho);
    ofstream saida("dataset_codificado_omp.csv");

    if (!entrada.is_open() || !saida.is_open()) {
        cerr << "Erro ao abrir arquivos." << endl;
        return;
    }

    string cabecalho;
    getline(entrada, cabecalho);
    saida << cabecalho << "\n";

    queue<pair<size_t, vector<string>>> fila_chunks;
    map<size_t, vector<string>> resultados;

    mutex mtx_fila, mtx_resultado;
    condition_variable cv_fila, cv_resultado;
    atomic<bool> leitura_concluida{false};

    size_t seq_id = 0;
    const unsigned N_CONSUMIDORES = thread::hardware_concurrency();
    vector<thread> consumidores;

    // CONSUMIDORES
    for (unsigned t = 0; t < N_CONSUMIDORES; ++t) {
        consumidores.emplace_back([&]() {
            while (true) {
                pair<size_t, vector<string>> item;

                {
                    unique_lock<mutex> lock(mtx_fila);
                    cv_fila.wait(lock, [&]() {
                        return !fila_chunks.empty() || leitura_concluida;
                    });

                    if (!fila_chunks.empty()) {
                        item = move(fila_chunks.front());
                        fila_chunks.pop();
                    } else if (leitura_concluida) {
                        break;
                    } else {
                        continue;
                    }
                }

                auto codificadas = codificar_linhas_chunk(item.second, nomeColunas, mapas);

                {
                    lock_guard<mutex> lock(mtx_resultado);
                    resultados[item.first] = move(codificadas);
                }

                cv_resultado.notify_one();
            }
        });
    }

    // ESCRITOR
    thread escritor([&]() {
        size_t proximo_seq = 0;

        while (true) {
            vector<string> linhas;
            {
                unique_lock<mutex> lock(mtx_resultado);
                cv_resultado.wait(lock, [&]() {
                    return resultados.count(proximo_seq) || (leitura_concluida && resultados.empty());
                });

                if (resultados.count(proximo_seq)) {
                    linhas = move(resultados[proximo_seq]);
                    resultados.erase(proximo_seq);
                    ++proximo_seq;
                } else if (leitura_concluida && resultados.empty()) {
                    break;
                } else {
                    continue;
                }
            }

            for (const auto& linha : linhas) {
                saida << linha << "\n";
            }
        }
    });

    // PRODUTOR (leitura do arquivo)
    string linha;
    vector<string> chunk;
    while (getline(entrada, linha)) {
        chunk.push_back(linha);
        if (chunk.size() >= linhas_por_chunk) {
            {
                lock_guard<mutex> lock(mtx_fila);
                fila_chunks.emplace(seq_id++, move(chunk));
            }
            cv_fila.notify_one();
            chunk.clear();
        }
    }

    if (!chunk.empty()) {
        {
            lock_guard<mutex> lock(mtx_fila);
            fila_chunks.emplace(seq_id++, move(chunk));
        }
        cv_fila.notify_one();
    }

    leitura_concluida = true;
    cv_fila.notify_all();
    cv_resultado.notify_all();

    for (auto& th : consumidores) th.join();
    escritor.join();

    entrada.close();
    saida.close();
}

#endif // NORMALIZADOR_OMP_HPP


Overwriting normalizador_omp.hpp


In [57]:
%%writefile main_omp.cpp

#include <iostream>
#include <vector>
#include <string>
#include <omp.h>
#include <unordered_map>
#include "leitura.hpp"
#include "sistema.hpp"
#include "benchmark.hpp"
#include "normalizador_omp.hpp"

constexpr const char* ARQUIVO_DATASET = "dataset_00_sem_virg.csv";

int main() {
    vector<string> nomeColunas;
    vector<int> indicesAlvo;
    vector<Linha> amostras;

    double inicio = omp_get_wtime();

    ler_cabecalho_e_amostras(ARQUIVO_DATASET, nomeColunas, amostras, 10);

    detectar_colunas_categoricas(nomeColunas, amostras, indicesAlvo);

    cout << "Quantidade de colunas categoricas detectadas: " << indicesAlvo.size() << endl;
    cout << "Colunas categoricas detectadas:" << endl;
    for (int indice : indicesAlvo) {
        if (indice < nomeColunas.size()) {
            cout << "- " << nomeColunas[indice] << endl;
        }
    }

    BenchmarkInfo info = coletar_info_benchmark(ARQUIVO_DATASET, amostras, indicesAlvo.size(), omp_get_max_threads());
    info.linhas_por_chunk = calcular_linhas_por_chunk(info.ram_disponivel_mb, info.bytes_por_linha);
    exibir_info_benchmark(info);

    unordered_map<string, unordered_map<string, int>> mapas;
    unordered_map<string, int> contadores;
    construir_mapas_codificacao_por_chunks(ARQUIVO_DATASET, nomeColunas, indicesAlvo, info.linhas_por_chunk, mapas, contadores);

    gerar_arquivos_id_por_coluna(mapas);
    gerar_dataset_codificado(ARQUIVO_DATASET, nomeColunas, indicesAlvo, mapas, info.linhas_por_chunk);

    double fim = omp_get_wtime();
    info.tempo_execucao = fim - inicio;

    salvar_info_benchmark_csv(info, "benchmark_omp.csv");

    cout << "Tempo de execucao: " << info.tempo_execucao << " segundos" << endl;
    cout << "Tempo de execucao formatado: " << formatar_tempo(info.tempo_execucao) << endl;
    return 0;
}

Overwriting main_omp.cpp


In [58]:
!g++ main_omp.cpp -fopenmp -std=c++17 -o main_omp.exe
!main_omp && del main_omp.exe

Quantidade de colunas categoricas detectadas: 12
Colunas categoricas detectadas:
- cdtup
- berco
- portoatracacao
- mes
- tipooperacao
- tiponavegacaoatracacao
- terminal
- origem
- destino
- cdmercadoria
- naturezacarga
- sentido

===== INFOS =====
Nome do arquivo: dataset_00_sem_virg.csv
Tamanho do arquivo: 2927 MB
RAM disponivel: 6292 MB
RAM total do sistema: 16330 MB
Bytes por linha (estimado): 222
Linhas por chunk: 1188763
Tamnho por chunk: 251 MB
Processador: AMD Ryzen 5 3600X 6-Core Processor             
Cores (threads): 12
Colunas categóricas detectadas: 12

Tempo de execucao: 248.027 segundos
Tempo de execucao formatado: 4m 8s


In [59]:
import os
import shutil
from glob import glob

# 1. Criar pasta de destino
destino = "omp_csv"
os.makedirs(destino, exist_ok=True)

# 2. Identificar arquivos CSV gerados
csvs = glob("*.csv")
csvs = [f for f in csvs if f.startswith("ID_") or f in ("dataset_codificado_omp.csv", "benchmark_omp.csv")]

# 3. Mover para pasta
for arquivo in csvs:
    shutil.move(arquivo, os.path.join(destino, arquivo))
    print(f"Movido: {arquivo} -> {destino}/")

if not csvs:
    print("Nenhum arquivo .csv correspondente encontrado.")


Movido: benchmark_omp.csv -> omp_csv/
Movido: dataset_codificado_omp.csv -> omp_csv/
Movido: ID_berco.csv -> omp_csv/
Movido: ID_cdmercadoria.csv -> omp_csv/
Movido: ID_cdtup.csv -> omp_csv/
Movido: ID_destino.csv -> omp_csv/
Movido: ID_mes.csv -> omp_csv/
Movido: ID_naturezacarga.csv -> omp_csv/
Movido: ID_origem.csv -> omp_csv/
Movido: ID_portoatracacao.csv -> omp_csv/
Movido: ID_sentido.csv -> omp_csv/
Movido: ID_terminal.csv -> omp_csv/
Movido: ID_tiponavegacaoatracacao.csv -> omp_csv/
Movido: ID_tipooperacao.csv -> omp_csv/


# Executar Threads

In [70]:
%%writefile normalizacao_threads.hpp

#ifndef NORMALIZACAO_THREADS_HPP
#define NORMALIZACAO_THREADS_HPP

#include <vector>
#include <string>
#include <unordered_map>
#include <unordered_set>
#include <fstream>
#include <sstream>
#include <memory>
#include <mutex>
#include <condition_variable>
#include <atomic>
#include <map>
#include <thread>
#include <queue>

using namespace std;

using Linha = vector<string>;


inline void construir_mapas_codificacao_por_chunks(const string& caminho,
                                                   const vector<string>& nomeColunas,
                                                   const vector<int>& indicesAlvo,
                                                   size_t linhas_por_chunk,
                                                   unordered_map<string, unordered_map<string, int>>& mapas,
                                                   unordered_map<string, int>& contadores) {
    ifstream arquivo(caminho);
    if (!arquivo.is_open()) {
        cerr << "Erro ao abrir o arquivo para processar chunks." << endl;
        exit(1);
    }

    string cabecalho;
    getline(arquivo, cabecalho); // descarta cabeçalho

    string linha;
    vector<Linha> chunk;
    mutex mtx;

    auto processar_chunk = [&](const vector<Linha>& chunk_local) {
        unsigned n_threads = thread::hardware_concurrency();
        if (n_threads == 0) n_threads = 2;

        vector<unordered_map<string, unordered_map<string, int>>> mapasLocais(n_threads);
        vector<thread> threads;

        for (unsigned tid = 0; tid < n_threads; ++tid) {
            threads.emplace_back([&, tid]() {
                for (size_t i = tid; i < indicesAlvo.size(); i += n_threads) {
                    int col = indicesAlvo[i];
                    string nomeCol = nomeColunas[col];
                    for (const auto& linhaChunk : chunk_local) {
                        if (col >= linhaChunk.size()) continue;
                        string valor = linhaChunk[col];
                        if (valor.empty()) continue;
                        mapasLocais[tid][nomeCol][valor];
                    }
                }
            });
        }

        for (auto& t : threads) t.join();

        lock_guard<mutex> lock(mtx);
        for (const auto& local : mapasLocais) {
            for (const auto& [coluna, mapa] : local) {
                for (const auto& [valor, _] : mapa) {
                    if (mapas[coluna].find(valor) == mapas[coluna].end()) {
                        mapas[coluna][valor] = contadores[coluna]++;
                    }
                }
            }
        }
    };

    while (getline(arquivo, linha)) {
        stringstream ss(linha);
        string valor;
        Linha linhaDados;
        while (getline(ss, valor, ',')) {
            linhaDados.push_back(valor);
        }
        chunk.push_back(linhaDados);

        if (chunk.size() >= linhas_por_chunk) {
            processar_chunk(chunk);
            chunk.clear();
        }
    }

    if (!chunk.empty()) {
        processar_chunk(chunk);
    }

    arquivo.close();
}

inline void gerar_arquivos_id_por_coluna(const unordered_map<string, unordered_map<string, int>>& mapas) {
    unsigned n_threads = thread::hardware_concurrency();
    if (n_threads == 0) n_threads = 2;

    vector<thread> threads;
    mutex mtx_erro;

    vector<string> chaves;
    for (const auto& [coluna, _] : mapas) {
        chaves.push_back(coluna);
    }

    auto worker = [&](unsigned tid) {
        for (size_t i = tid; i < chaves.size(); i += n_threads) {
            const string& nomeCol = chaves[i];
            const auto& mapa = mapas.at(nomeCol);

            ofstream arquivo("ID_" + nomeCol + ".csv");
            if (!arquivo.is_open()) {
                lock_guard<mutex> lock(mtx_erro);
                cerr << "Erro ao criar arquivo ID_" << nomeCol << ".csv" << endl;
                continue;
            }

            arquivo << "COD,CONTEUDO\n";
            for (const auto& [conteudo, cod] : mapa) {
                arquivo << cod << "," << conteudo << "\n";
            }

            arquivo.close();
        }
    };

    for (unsigned tid = 0; tid < n_threads; ++tid) {
        threads.emplace_back(worker, tid);
    }

    for (auto& t : threads) t.join();
}

inline vector<string> codificar_linhas_chunk(const vector<string>& chunk,
                                             const vector<string>& nomeColunas,
                                             const unordered_map<string, unordered_map<string, int>>& mapas) {
    vector<string> linhasCodificadas(chunk.size());

    unsigned n_threads = thread::hardware_concurrency();
    if (n_threads == 0) n_threads = 2;
    vector<thread> threads;

    size_t linhas_por_thread = (chunk.size() + n_threads - 1) / n_threads;

    for (unsigned tid = 0; tid < n_threads; ++tid) {
        threads.emplace_back([&, tid]() {
            size_t inicio = tid * linhas_por_thread;
            size_t fim = min(inicio + linhas_por_thread, chunk.size());

            for (size_t i = inicio; i < fim; ++i) {
                stringstream ss(chunk[i]);
                string valor;
                vector<string> campos;
                while (getline(ss, valor, ',')) {
                    campos.push_back(valor);
                }

                stringstream codificada;
                for (size_t j = 0; j < campos.size(); j++) {
                    const string& nomeCol = nomeColunas[j];
                    if (mapas.count(nomeCol)) {
                        codificada << mapas.at(nomeCol).at(campos[j]);
                    } else {
                        codificada << campos[j];
                    }
                    if (j < campos.size() - 1) codificada << ",";
                }

                linhasCodificadas[i] = codificada.str();
            }
        });
    }

    for (auto& t : threads) t.join();

    return linhasCodificadas;
}

inline void gerar_dataset_codificado(const string& caminho,
                                     const vector<string>& nomeColunas,
                                     const vector<int>& indicesAlvo,
                                     const unordered_map<string, unordered_map<string, int>>& mapas,
                                     size_t linhas_por_chunk) {
    ifstream entrada(caminho);
    ofstream saida("dataset_codificado_th.csv");

    if (!entrada.is_open() || !saida.is_open()) {
        cerr << "Erro ao abrir arquivos." << endl;
        return;
    }

    string cabecalho;
    getline(entrada, cabecalho);
    saida << cabecalho << "\n";

    queue<pair<size_t, vector<string>>> fila_chunks;
    map<size_t, vector<string>> resultados;

    mutex mtx_fila, mtx_resultado;
    condition_variable cv_fila, cv_resultado;
    atomic<bool> leitura_concluida{false};

    size_t seq_id = 0;
    const unsigned N_CONSUMIDORES = thread::hardware_concurrency();
    vector<thread> consumidores;

    // CONSUMIDORES
    for (unsigned t = 0; t < N_CONSUMIDORES; ++t) {
        consumidores.emplace_back([&]() {
            while (true) {
                pair<size_t, vector<string>> item;

                {
                    unique_lock<mutex> lock(mtx_fila);
                    cv_fila.wait(lock, [&]() {
                        return !fila_chunks.empty() || leitura_concluida;
                    });

                    if (!fila_chunks.empty()) {
                        item = move(fila_chunks.front());
                        fila_chunks.pop();
                    } else if (leitura_concluida) {
                        break;
                    } else {
                        continue;
                    }
                }

                auto codificadas = codificar_linhas_chunk(item.second, nomeColunas, mapas);

                {
                    lock_guard<mutex> lock(mtx_resultado);
                    resultados[item.first] = move(codificadas);
                }

                cv_resultado.notify_one();
            }
        });
    }

    // ESCRITOR
    thread escritor([&]() {
        size_t proximo_seq = 0;

        while (true) {
            vector<string> linhas;
            {
                unique_lock<mutex> lock(mtx_resultado);
                cv_resultado.wait(lock, [&]() {
                    return resultados.count(proximo_seq) || (leitura_concluida && resultados.empty());
                });

                if (resultados.count(proximo_seq)) {
                    linhas = move(resultados[proximo_seq]);
                    resultados.erase(proximo_seq);
                    ++proximo_seq;
                } else if (leitura_concluida && resultados.empty()) {
                    break;
                } else {
                    continue;
                }
            }

            for (const auto& linha : linhas) {
                saida << linha << "\n";
            }
        }
    });

    // PRODUTOR (leitura do arquivo)
    string linha;
    vector<string> chunk;
    while (getline(entrada, linha)) {
        chunk.push_back(linha);
        if (chunk.size() >= linhas_por_chunk) {
            {
                lock_guard<mutex> lock(mtx_fila);
                fila_chunks.emplace(seq_id++, move(chunk));
            }
            cv_fila.notify_one();
            chunk.clear();
        }
    }

    if (!chunk.empty()) {
        {
            lock_guard<mutex> lock(mtx_fila);
            fila_chunks.emplace(seq_id++, move(chunk));
        }
        cv_fila.notify_one();
    }

    leitura_concluida = true;
    cv_fila.notify_all();
    cv_resultado.notify_all();

    for (auto& th : consumidores) th.join();
    escritor.join();

    entrada.close();
    saida.close();
}

#endif // NORMALIZACAO_THREADS_HPP

Overwriting normalizacao_threads.hpp


In [71]:
%%writefile main_threads.cpp

#include <iostream>
#include <vector>
#include <string>
#include <omp.h>
#include <unordered_map>
#include "leitura.hpp"
#include "benchmark.hpp"
#include "normalizacao_threads.hpp"

constexpr const char* ARQUIVO_DATASET = "dataset_00_sem_virg.csv";

using namespace std;

int main() {
    vector<string> nomeColunas;
    vector<int> indicesAlvo;
    vector<Linha> amostras;
    int linhas_amostra = 10;

    double inicio = omp_get_wtime();

    ler_cabecalho_e_amostras(ARQUIVO_DATASET, nomeColunas, amostras, linhas_amostra);
    
    detectar_colunas_categoricas(nomeColunas, amostras, indicesAlvo);

    cout << "Quantidade de colunas categoricas detectadas: " << indicesAlvo.size() << endl;
    cout << "Colunas categoricas detectadas:" << endl;
    for (int indice : indicesAlvo) {
        if (indice < nomeColunas.size()) {
            cout << "- " << nomeColunas[indice] << endl;
        }
    }

    BenchmarkInfo info = coletar_info_benchmark(ARQUIVO_DATASET, amostras, indicesAlvo.size(), thread::hardware_concurrency());
    info.linhas_por_chunk = calcular_linhas_por_chunk(info.ram_disponivel_mb, info.bytes_por_linha);
    exibir_info_benchmark(info);

    unordered_map<string, unordered_map<string, int>> mapas;
    unordered_map<string, int> contadores;
    construir_mapas_codificacao_por_chunks(ARQUIVO_DATASET, nomeColunas, indicesAlvo, info.linhas_por_chunk, mapas, contadores);

    gerar_arquivos_id_por_coluna(mapas);
    gerar_dataset_codificado(ARQUIVO_DATASET, nomeColunas, indicesAlvo, mapas, info.linhas_por_chunk);

    double fim = omp_get_wtime();
    info.tempo_execucao = fim - inicio;

    cout << "Tempo de execucao: " << info.tempo_execucao << " segundos" << endl;
    cout << "Tempo de execucao formatado: " << formatar_tempo(info.tempo_execucao) << endl;
    
    salvar_info_benchmark_csv(info, "benchmark_threads.csv");
    
    return 0;
}


Overwriting main_threads.cpp


In [72]:
!g++ -std=c++17 main_threads.cpp -o main_th.exe -fopenmp
!main_th && del main_th.exe

Quantidade de colunas categoricas detectadas: 12
Colunas categoricas detectadas:
- cdtup
- berco
- portoatracacao
- mes
- tipooperacao
- tiponavegacaoatracacao
- terminal
- origem
- destino
- cdmercadoria
- naturezacarga
- sentido

===== INFOS =====
Nome do arquivo: dataset_00_sem_virg.csv
Tamanho do arquivo: 2927 MB
RAM disponivel: 5930 MB
RAM total do sistema: 16330 MB
Bytes por linha (estimado): 222
Linhas por chunk: 1120370
Tamnho por chunk: 237 MB
Processador: AMD Ryzen 5 3600X 6-Core Processor             
Cores (threads): 12
Colunas categóricas detectadas: 12

Tempo de execucao: 248.989 segundos
Tempo de execucao formatado: 4m 8s


In [73]:
import os
import shutil
from glob import glob

# 1. Criar pasta de destino
destino = "thread_csv"
os.makedirs(destino, exist_ok=True)

# 2. Identificar arquivos CSV gerados
csvs = glob("*.csv")
csvs = [f for f in csvs if f.startswith("ID_") or f in ("dataset_codificado_th.csv", "benchmark_threads.csv")]

# 3. Mover para pasta
for arquivo in csvs:
    shutil.move(arquivo, os.path.join(destino, arquivo))
    print(f"Movido: {arquivo} -> {destino}/")

if not csvs:
    print("Nenhum arquivo .csv correspondente encontrado.")


Movido: benchmark_threads.csv -> thread_csv/
Movido: dataset_codificado_th.csv -> thread_csv/
Movido: ID_berco.csv -> thread_csv/
Movido: ID_cdmercadoria.csv -> thread_csv/
Movido: ID_cdtup.csv -> thread_csv/
Movido: ID_destino.csv -> thread_csv/
Movido: ID_mes.csv -> thread_csv/
Movido: ID_naturezacarga.csv -> thread_csv/
Movido: ID_origem.csv -> thread_csv/
Movido: ID_portoatracacao.csv -> thread_csv/
Movido: ID_sentido.csv -> thread_csv/
Movido: ID_terminal.csv -> thread_csv/
Movido: ID_tiponavegacaoatracacao.csv -> thread_csv/
Movido: ID_tipooperacao.csv -> thread_csv/


# Comparação OMP x Threads

In [74]:
# Ensure pandas and Jinja2 are installed
try:
    import pandas as pd
except ModuleNotFoundError:
    import os
    os.system("pip install pandas")
    import pandas as pd

try:
    import jinja2
except ModuleNotFoundError:
    import os
    os.system("pip install Jinja2")
    import jinja2

# Carregar CSVs atualizados
df_omp = pd.read_csv("omp_csv/benchmark_omp.csv")
df_thread = pd.read_csv("thread_csv/benchmark_threads.csv")

# Marcar a origem
df_omp["Tipo"] = "OMP"
df_thread["Tipo"] = "Threads"

# Unir
df = pd.concat([df_omp, df_thread], ignore_index=True)

# Renomear colunas para apresentação
df = df.rename(columns={
    "Tipo": "Tipo",
    "nome_arquivo": "Arquivo",
    "nome_cpu": "CPU",
    "num_threads": "Threads",
    "ram_total_mb": "RAM Total",
    "ram_disponivel_mb": "RAM Disponível",
    "tamanho_arq_mb": "Tamanho Arquivo",
    "bytes_por_linha": "Bytes por Linha",
    "linhas_por_chunk": "Linhas por Chunk",
    "tamanho_chunk_mb": "Tamanho do Chunk (MB)",
    "total_col_categoricas": "Colunas Categóricas",
    "tempo_execucao_segundos": "Tempo (s)",
    "tempo_formatado": "Tempo Formatado",
})

# Ordenar colunas com Tipo no início
colunas = [
    "Tipo", "Arquivo", "CPU", "Threads",
    "RAM Total", "RAM Disponível", "Tamanho Arquivo",
    "Bytes por Linha", "Linhas por Chunk", "Tamanho do Chunk (MB)",
    "Colunas Categóricas", "Tempo (s)", "Tempo Formatado"
]
df = df[colunas]

# Formatando colunas com MB
for col in ["RAM Total", "RAM Disponível", "Tamanho Arquivo", "Tamanho do Chunk (MB)"]:
    df[col] = df[col].map(lambda x: f"{x:.0f} MB" if "Chunk" not in col else f"{x:.2f} MB")

# Mostrar com estilo
df.style.set_caption("Benchmark Comparativo: OMP vs Threads") \
    .set_table_attributes("style='display:inline'") \
    .format({
        "Bytes por Linha": "{:,}",
        "Linhas por Chunk": "{:,}",
        "Colunas Categóricas": "{}",
        "Tempo (s)": "{:.2f} s"
    })


,Tipo,Arquivo,CPU,Threads,RAM Total,RAM Disponível,Tamanho Arquivo,Bytes por Linha,Linhas por Chunk,Tamanho do Chunk (MB),Colunas Categóricas,Tempo (s),Tempo Formatado
0,OMP,dataset_00_sem_virg.csv,AMD Ryzen 5 3600X 6-Core Processor,12,16330 MB,6292 MB,2927 MB,222,"1,188,763",251.00 MB,12,248.03 s,4m 8s
1,Threads,dataset_00_sem_virg.csv,AMD Ryzen 5 3600X 6-Core Processor,12,16330 MB,5930 MB,2927 MB,222,"1,120,370",237.00 MB,12,248.99 s,4m 8s


# Desafios

* Leitura eficiente de grandes arquivos CSV
  - Arquivos com milhões de linhas não podem ser carregados inteiramente na RAM.
  - Solução: leitura por chunks, baseada em estimativa da memória disponível e do tamanho médio por linha.
* Detecção automática de colunas categóricas
  - Precisava distinguir colunas numéricas de categóricas de forma dinâmica.
  - Solução: análise das 5 primeiras linhas e exclusão de campos nulos e numéricos.
* Normalização categórica com paralelismo
  - Codificação de valores únicos por coluna sem travamento entre threads.
  - Solução: uso de mapas locais dentro de cada thread durante a contagem de valores por coluna, seguido por um bloco #pragma omp critical apenas no momento da fusão dos dados no mapa global.

# Possíveis Melhorias

* Evitar a alocação dinâmica excessiva das bibliotecas padrão do C++ adotando buffers pré-alocados e estruturas otimizadas para uso intensivo de memória.
* Monitoramento do uso de memória RAM durante a execução do programa